In [54]:
import sqlalchemy as sa
import pandas as pd

In [20]:
eng = sa.create_engine('vertica+vertica_python://dbadmin:@localhost:5433/VMart')

### Dictionary tables

In [35]:
activity_type_file = "../../sandbox.KNU/dictionary/activity_type.tsv"

activity_type_df = pd.read_csv(activity_type_file, sep='\t')
activity_type_df.to_sql("activity_type", 
                        if_exists="replace",
                        con=eng,
                        schema="dictionary",
                        index=False,
                        dtype={
                            'id': sa.types.INTEGER(),
                            'type': sa.types.VARCHAR(50)
                        })


In [34]:
context_file = "../../sandbox.KNU/dictionary/context.tsv"

context_df = pd.read_csv(context_file, sep='\t')
context_df.to_sql("context", 
                        if_exists="replace",
                        con=eng,
                        schema="dictionary",
                        index=False,
                        dtype={
                            'id': sa.types.INTEGER(),
                            'name': sa.types.VARCHAR(50)
                        })


In [41]:
service_type_file = "../../sandbox.KNU/dictionary/service_type.tsv"

service_type_df = pd.read_csv(service_type_file, sep='\t')
service_type_df.to_sql("service_type", 
                        if_exists="replace",
                        con=eng,
                        schema="dictionary",
                        index=False,
                        dtype={
                            'type_id': sa.types.INTEGER(),
                            'name': sa.types.VARCHAR(50)
                        })


### Marketing tables

#### Orders table

In [45]:
orders_file = "../../sandbox.KNU/marketing/orders.tsv"

orders_df = pd.read_csv(orders_file, sep='\t')
orders_df

,order_id,dt,user_id,gross,type_id,context_id,parent_order_id
0,31954283,2020-06-28 05:43:18.000000,1516604161,1.98,3,4,NaN
1,31954691,2020-06-28 06:26:22.000000,1516459566,1.98,3,6,NaN
2,31955380,2020-06-28 07:30:07.000000,1516791409,1.98,3,7,NaN
3,31955397,2020-06-28 07:30:27.000000,1516791409,4.60,21,5,NaN
4,31956537,2020-06-28 09:07:30.000000,1516891400,1.98,3,4,NaN
...,...,...,...,...,...,...,...
18617,34439212,2020-10-28 23:20:08.000000,1797239543,15.23,21,5,NaN
18618,34439382,2020-10-28 23:29:01.000000,1787559126,32.12,3,1,34351603.0
18619,34439356,2020-10-28 23:30:12.000000,1777954113,1.98,3,7,NaN
18620,34439782,2020-10-28 23:49:03.000000,1802210881,17.98,3,7,NaN


As we can see, in `parent_order_id` column there are non integers, but they are supposed to be. So we need to convert them before uploading

In [52]:
orders_df['parent_order_id'] = orders_df['parent_order_id'].astype("Int32")

In [53]:
orders_df

,order_id,dt,user_id,gross,type_id,context_id,parent_order_id
0,31954283,2020-06-28 05:43:18.000000,1516604161,1.98,3,4,<NA>
1,31954691,2020-06-28 06:26:22.000000,1516459566,1.98,3,6,<NA>
2,31955380,2020-06-28 07:30:07.000000,1516791409,1.98,3,7,<NA>
3,31955397,2020-06-28 07:30:27.000000,1516791409,4.60,21,5,<NA>
4,31956537,2020-06-28 09:07:30.000000,1516891400,1.98,3,4,<NA>
...,...,...,...,...,...,...,...
18617,34439212,2020-10-28 23:20:08.000000,1797239543,15.23,21,5,<NA>
18618,34439382,2020-10-28 23:29:01.000000,1787559126,32.12,3,1,34351603
18619,34439356,2020-10-28 23:30:12.000000,1777954113,1.98,3,7,<NA>
18620,34439782,2020-10-28 23:49:03.000000,1802210881,17.98,3,7,<NA>


In [55]:
orders_df.to_sql("orders", 
                        if_exists="replace",
                        con=eng,
                        schema="marketing",
                        index=False,
                        dtype={
                            'order_id': sa.types.INTEGER(),
                            'dt': sa.DateTime(),
                            'user_id': sa.types.INTEGER(),
                            'gross': sa.types.FLOAT(),
                            'type_id': sa.types.INTEGER(),
                            'context_id': sa.types.INTEGER(),
                            'parent_order_id': sa.types.INTEGER()
                        })


#### Subchannels cost table

In [62]:
subchannels_cost_file = "../../sandbox.KNU/marketing/subchannels_cost.tsv"

subchannels_cost_df = pd.read_csv(subchannels_cost_file, sep='\t')
subchannels_cost_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195126 entries, 0 to 195125
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   subchannel_id  195126 non-null  int64  
 1   dt             195126 non-null  object 
 2   cost           195126 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 4.5+ MB


Zero id looks a bit strange, I am not sure here is it missing or there is really zero-indexed subchannel, so will leave it alive

In [68]:
subchannels_cost_df['subchannel_id'].describe()

count    1.951260e+05
mean     1.412921e+08
std      5.071496e+07
min      0.000000e+00
25%      1.099928e+08
50%      1.171573e+08
75%      1.658541e+08
max      2.566522e+08
Name: subchannel_id, dtype: float64

In [66]:
subchannels_cost_df.to_sql("subchannels_cost", 
                        if_exists="replace",
                        con=eng,
                        schema="marketing",
                        index=False,
                        dtype={
                            'subchannel_id': sa.types.INTEGER(),
                            'dt': sa.DateTime(),
                            'cost': sa.types.FLOAT()
                        })


#### Subscription status table

In [70]:
subscription_status_file = "../../sandbox.KNU/marketing/subscription_status.tsv"

subscription_status_df = pd.read_csv(subscription_status_file, sep='\t')
subscription_status_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15753 entries, 0 to 15752
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   parent_order_id  15753 non-null  int64 
 1   dt               15753 non-null  object
 2   status           15753 non-null  object
dtypes: int64(1), object(2)
memory usage: 369.3+ KB


In [73]:
subscription_status_df.isnull().sum(axis=1)

0        0
1        0
2        0
3        0
4        0
        ..
15748    0
15749    0
15750    0
15751    0
15752    0
Length: 15753, dtype: int64

In [75]:
subscription_status_df.to_sql("subscription_status", 
                        if_exists="replace",
                        con=eng,
                        schema="marketing",
                        index=False,
                        dtype={
                            'status': sa.types.VARCHAR(50),
                            'dt': sa.DateTime(),
                            'parent_order_id': sa.types.INTEGER()
                        })


### Product tables

#### Activities table

In [78]:
activities_file = "../../sandbox.KNU/product/activities.tsv"

activities_df = pd.read_csv(activities_file, sep='\t')
activities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204514 entries, 0 to 204513
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   dt             204514 non-null  object
 1   activity_type  204514 non-null  int64 
 2   user_id        204514 non-null  int64 
 3   contact_id     204514 non-null  int64 
 4   user_app       204514 non-null  object
dtypes: int64(3), object(2)
memory usage: 7.8+ MB


In [79]:
activities_df.to_sql("activities", 
                        if_exists="replace",
                        con=eng,
                        schema="product",
                        index=False,
                        dtype={
                            'dt': sa.DateTime(),
                            'activity_type': sa.types.INTEGER(),
                            'user_id': sa.types.INTEGER(),
                            'contact_id': sa.types.INTEGER(),
                            'user_app': sa.types.VARCHAR(50)
                        })


#### Users table

In [81]:
users_file = "../../sandbox.KNU/product/users.tsv"

users_df = pd.read_csv(users_file, sep='\t')
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1013277 entries, 0 to 1013276
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   reg_dt         1013277 non-null  object
 1   id             1013277 non-null  int64 
 2   gender         1013277 non-null  object
 3   age            1013277 non-null  int64 
 4   country_code   1013075 non-null  object
 5   device_type    1013277 non-null  int64 
 6   app            1013277 non-null  object
 7   lang           1013277 non-null  object
 8   subchannel_id  1013277 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 69.6+ MB


In [83]:
users_df.to_sql("users", 
                        if_exists="replace",
                        con=eng,
                        schema="product",
                        index=False,
                        dtype={
                            'reg_dt': sa.DateTime(),
                            'id': sa.types.INTEGER(),
                            'age': sa.types.INTEGER(),
                            'device_type': sa.types.INTEGER(),
                            'subchannel_id': sa.types.INTEGER(),
                            'country_code': sa.types.VARCHAR(2),
                            'lang': sa.types.VARCHAR(2),
                            'gender': sa.types.VARCHAR(1),
                            'app': sa.types.VARCHAR(20),
                        })


#### User session end table

In [87]:
user_sesson_end_file = "../../sandbox.KNU/product/user_session_end.tsv"

user_session_end_df = pd.read_csv(user_sesson_end_file, sep='\t')
user_session_end_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2978202 entries, 0 to 2978201
Data columns (total 5 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   user_id         int64 
 1   dt              object
 2   duration        int64 
 3   cnt_visitors    int64 
 4   session_number  int64 
dtypes: int64(4), object(1)
memory usage: 113.6+ MB


In [88]:
user_session_end_df.to_sql("user_session_end", 
                        if_exists="replace",
                        con=eng,
                        schema="product",
                        index=False,
                        dtype={
                            'dt': sa.DateTime(),
                            'user_id': sa.types.INTEGER(),
                            'duration': sa.types.INTEGER(),
                            'cnt_visitors': sa.types.INTEGER(),
                            'session_number': sa.types.INTEGER(),
                        })
